# Random Strategy

Start off with a random strategy of choosing which transactions and cards to open

In [1]:
import pandas as pd
import numpy as np
import json
from collections import OrderedDict 

from Card import Card
from User import User

In [2]:
# read in data
transactions = pd.read_csv('transactions_clean.csv')
with open('card_options.json', 'r') as f:
    card_options = json.load(f)

In [3]:
cards = card_options['cards']

In [4]:
transactions['category'].value_counts()

travel           24
unclassified     24
dining           24
grocery          23
entertainment    18
drug store       15
Name: category, dtype: int64

In [5]:
# next steps: 
# - make sure transactions df has category as a column and is aggregated by month
# - run simulation, where i pick a random action for each step
# - also need to think of how i know when i need to increment the time.  Maybe that's part of the transactions??

In [51]:
state_values = {}
# add cards 
for a1 in [0,1]:
    for a2 in [0,1]:    
        for a3 in [0,1]:    
            for a4 in [0,1]:    
                for a5 in [0,1]:    
                    for a6 in [0,1]:    
                        for a7 in [0,1]:    
                            for a8 in [0,1]:  
                                for m in range(24):
                                    for c in set(transactions['category']):
                                        state_values[(a1, a2, a3, a4, a5, a6, a7, a8, m, c)] = 0

In [72]:
def access_state(sv, step):
    k1 = step['Chase Freedom Unlimited']
    k2 = step['Chase Freedom Flex']
    k3 = step['Chase Sapphire Preferred']
    k4 = step['Chase Sapphire Reserve']
    k5 = step['Capital One Savor']
    k6 = step['Capital One SavorOne']
    k7 = step['Capital One Quicksilver']
    k8 = step['Capital One Venture']
    k9 = step['month']
    k10 = step['category']
    return sv[(k1, k2, k3, k4, k5, k6, k7, k8, k9, k10)]

def update_values(sv, step, value):
    k1 = step['Chase Freedom Unlimited']
    k2 = step['Chase Freedom Flex']
    k3 = step['Chase Sapphire Preferred']
    k4 = step['Chase Sapphire Reserve']
    k5 = step['Capital One Savor']
    k6 = step['Capital One SavorOne']
    k7 = step['Capital One Quicksilver']
    k8 = step['Capital One Venture']
    k9 = step['month']
    k10 = step['category']
    
    sv[(k1, k2, k3, k4, k5, k6, k7, k8, k9, k10)] = sv[(k1, k2, k3, k4, k5, k6, k7, k8, k9, k10)] + .25 * value
    


In [85]:
for n in range(1000):
    cuml_value = 0    
    user = User(cards, transactions)
    step_list = []
    step_list.append(user.state)
    step = user.state
    done = user.done
    while done == False:
        actions = user.next_possible_actions()
        max_reward = None
        action = None
        for a in actions:
            r = access_state(state_values, step)
            if max_reward is None:
                max_rewards = r
                action = a
            elif max_reward < r:
                max_reward = r
                action = a
                
#         action = actions[np.random.randint(0,len(actions))]

        step, reward, done = user.step(actions[0])
        cuml_value += reward
        if done:
            break
        step_list.append(step.copy())

#             print (step)
#         print (step)
    for i in range(1, len(step_list)-1):
        update_values(state_values, step_list[i], cuml_value)

In [86]:
print(np.min(list(state_values.values())))
print(np.max(list(state_values.values())))
print(np.mean(list(state_values.values())))

-9762449.764312608
0.0
-33367.748217865344


# next steps
- probably need to reduce states or use a neural network
- probably need to use something more complex than a monte carlo, q-learning seems more relevant

In [88]:
len(state_values)

36864